# Выбор локации для скважины

Нужно решить, где добывающей компании «ГлавРосГосНефть» бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски.



**Цели исследования:**
Построение модели для определения региона, где добыча принесёт наибольшую прибыль.

**Описание данных:** Датасеты с информацией о геологоразведке трех регионов: три признака точек и объемы запасов в скважине.

**План работы:**

1 ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ

1.1 Импортируем необходимые библиотеки, считаем данные из csv-файлов в датафреймы, сохраним в переменные и выведем их и основную информацию о них на экран

1.2 Проверим датасеты на наличие дубликатов

1.3 Выделим признаки и целевые признаки

2 ОБУЧЕНИЕ И ПРОВЕРКА МОДЕЛИ

2.1 Обучение модели для первого региона

2.2 Обучение модели для второго региона

2.3 Обучение модели для третьего региона

3 ПОДГОТОВКА К РАСЧЕТУ ПРИБЫЛИ

3.1 Сохраним ключевые значения для расчетов в отдельных переменных

3.2 Рассчитаем достаточный объем сырья для безубыточной разработки новой скважины

4 РАСЧЕТ ПРИБЫЛИ ПО ВЫБРАННЫМ СКВАЖИНАМ И ПРЕДСКАЗАНИЯМ МОДЕЛИ

4.1 Напишем функцию для расчета прибыли

4.2 Рассчитаем прибыль для каждого региона

5 РАСЧЕТ ПРИБЫЛИ И РИСКОВ

6 ОБЩИЙ ВЫВОД

## Загрузка и подготовка данных

**1.1 Импортируем необходимые библиотеки, считаем данные из csv-файлов в датафреймы, сохраним в переменные и выведем их и основную информацию о них на экран**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [2]:
data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [3]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**1.2 Проверим датасеты на наличие дубликатов**

In [9]:
data_0.duplicated().sum()

0

In [10]:
data_1.duplicated().sum()

0

In [11]:
data_2.duplicated().sum()

0

**1.3 Выделим признаки и целевые признаки**

In [12]:
target_0 = data_0['product']

# Также удалим ненужный столбец id:
features_0 = data_0.drop(['id', 'product'], axis=1)

In [13]:
target_1 = data_1['product']
features_1 = data_1.drop(['id', 'product'], axis=1)

In [14]:
target_2 = data_2['product']
features_2 = data_2.drop(['id', 'product'], axis=1)

**Вывод:** В ходе подготовки данных датасеты были проверены на наличие пропусков и дубликатов; удален столбец id; выделены признаки и целевой признак.

## Обучение и проверка модели

**2.1 Обучение модели для первого региона**

In [15]:
# Разделим датасет на обучающую и валидационную выборки:

features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(features_0, target_0, test_size=0.25, random_state=12345)

In [16]:
# Инициализируем модель LinearRegression:
model = LinearRegression()

# Обучим модель на тренировочной выборке:
model.fit(features_0_train, target_0_train)

# Получим предсказания модели на валидационной выборке:
predictions_0_valid = model.predict(features_0_valid)

In [17]:
# Преобразуем полученные предсказания в Series:
predictions_0_valid = pd.Series(predictions_0_valid)

# И выведем на экран:
predictions_0_valid

0         95.894952
1         77.572583
2         77.892640
3         90.175134
4         70.510088
            ...    
24995    103.037104
24996     85.403255
24997     61.509833
24998    118.180397
24999    118.169392
Length: 25000, dtype: float64

In [18]:
# Перенумеруем и выведем на экран правильные ответы на валидационной выборке:

target_0_valid = target_0_valid.reset_index(drop=True)
target_0_valid

0         10.038645
1        114.551489
2        132.603635
3        169.072125
4        122.325180
            ...    
24995    170.116726
24996     93.632175
24997    127.352259
24998     99.782700
24999    177.821022
Name: product, Length: 25000, dtype: float64

In [19]:
# Определим средний запас предсказанного сырья:

predictions_0_valid.mean()

92.59256778438035

In [20]:
# Посчитаем значение метрики RMSE на валидационной выборке:

print("RMSE модели линейной регрессии на валидационной выборке 0:", mean_squared_error(target_0_valid, predictions_0_valid)**0.5)

RMSE модели линейной регрессии на валидационной выборке 0: 37.5794217150813


**2.2 Обучение модели для второго региона**

In [21]:
features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(features_1, target_1, test_size=0.25, random_state=12345)

In [22]:
model = LinearRegression()
model.fit(features_1_train, target_1_train)
predictions_1_valid = model.predict(features_1_valid)

In [23]:
predictions_1_valid = pd.Series(predictions_1_valid)
predictions_1_valid

0         82.663314
1         54.431786
2         29.748760
3         53.552133
4          1.243856
            ...    
24995    136.869211
24996    110.693465
24997    137.879341
24998     83.761966
24999     53.958466
Length: 25000, dtype: float64

In [24]:
target_1_valid = target_1_valid.reset_index(drop=True)
target_1_valid

0         80.859783
1         53.906522
2         30.132364
3         53.906522
4          0.000000
            ...    
24995    137.945408
24996    110.992147
24997    137.945408
24998     84.038886
24999     53.906522
Name: product, Length: 25000, dtype: float64

In [25]:
predictions_1_valid.mean()

68.728546895446

In [26]:
print("RMSE модели линейной регрессии на валидационной выборке 1:", mean_squared_error(target_1_valid, predictions_1_valid)**0.5)

RMSE модели линейной регрессии на валидационной выборке 1: 0.893099286775617


**2.3 Обучение модели для третьего региона**

In [27]:
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

In [28]:
model = LinearRegression()
model.fit(features_2_train, target_2_train)
predictions_2_valid = model.predict(features_2_valid)

In [29]:
predictions_2_valid

array([ 93.59963303,  75.10515854,  90.06680936, ...,  99.40728116,
        77.77991248, 129.03241718])

In [30]:
predictions_2_valid = pd.Series(predictions_2_valid)
predictions_2_valid

0         93.599633
1         75.105159
2         90.066809
3        105.162375
4        115.303310
            ...    
24995     78.765887
24996     95.603394
24997     99.407281
24998     77.779912
24999    129.032417
Length: 25000, dtype: float64

In [31]:
target_2_valid = target_2_valid.reset_index(drop=True)
target_2_valid

0         61.212375
1         41.850118
2         57.776581
3        100.053761
4        109.897122
            ...    
24995     28.492402
24996     21.431303
24997    125.487229
24998     99.422903
24999    127.445075
Name: product, Length: 25000, dtype: float64

In [32]:
predictions_2_valid.mean()

94.96504596800489

In [33]:
print("RMSE модели линейной регрессии на валидационной выборке 2:", mean_squared_error(target_2_valid, predictions_2_valid)**0.5)

RMSE модели линейной регрессии на валидационной выборке 2: 40.02970873393434


**Вывод:** В результате обучения модели на валидационной выборке, самый высокий средний запас предсказанного сырья демонстрирует третий регион. Его показатель составил 94,97 тысяч баррелей. Корень из средней квадратичной ошибки - примерно 40 тысяч баррелей. Немного отстает регион номер один: 92,59 тысяч. RMSE около 38-ми. При максимальном значении среднего запаса около 165-ти тысяч баррелей, это выглядит правдоподобно. Но второй регион с самым низким предсказанным сырьем 68,73 тысячи баррелей показывает корень из средней квадратичной ошибки всего 0,89. Возможно, стоит обратить на него внимание.

## Подготовка к расчёту прибыли

**3.1 Сохраним ключевые значения для расчетов в отдельных переменных**

In [34]:
BUDGET = 10000000000
WELL_NUMBER = 200
UNIT_PROFIT = 450000

**3.2 Рассчитаем достаточный объем сырья для безубыточной разработки новой скважины**

In [35]:
 volume_min = (BUDGET / WELL_NUMBER) / UNIT_PROFIT
 volume_min

111.11111111111111

In [36]:
# Определим средний запас сырья в каждом регионе:

target_0.mean()

92.50000000000001

In [37]:
target_1.mean()

68.82500000000002

In [38]:
target_2.mean()

95.00000000000004

**Вывод:** Достаточный объем сырья для безубыточной разработки новой скважины составляет около 111 тысяч баррелей. Средние запасы каждого региона ниже этого показателя. Ближе всего к значению подходят первый (92,5 тысячи) и третий (95 тысяч) регионы.

## Расчёт прибыли по выбранным скважинам и предсказаниям модели

**4.1 Напишем функцию для расчета прибыли**

In [39]:
def profits(target, predictions, budget, unit_profit):
  preds_sorted = predictions.sort_values(ascending=True)
  selected = target[preds_sorted.index]
  return (selected.tail(200).sum() * unit_profit - budget).round(1)

**4.2 Рассчитаем прибыль для каждого региона**

In [40]:
profits(target_0_valid, predictions_0_valid, BUDGET, UNIT_PROFIT)

3320826043.1

In [41]:
profits(target_1_valid, predictions_1_valid, BUDGET, UNIT_PROFIT)

2415086696.7

In [42]:
profits(target_2_valid, predictions_2_valid, BUDGET, UNIT_PROFIT)

2710349963.6

**Вывод:** Создана функция для расчета прибыли по выбранным скважинам и предсказаниям модели. Произведен расчет для всех регионов. Наибольшую расчетную прибыль в размере примерно 3,3 миллиарда рублей показывает первый регион. Регион номер три на втором месте (2,7 миллиарда). У региона номер два только 2,4 миллиарда.

## Расчёт прибыли и рисков

In [43]:
state = np.random.RandomState(12345)

# Создадим пустой список для искомого значения:
values = []

# Создадим цикл для тысячи выборок:
for i in range(1000):
    # Разделим целевой признак и предсказания на подвыборки:
    target_subsample = target_0_valid.sample(n=500, replace=True, random_state=state)
    preds_subsample = predictions_0_valid[target_subsample.index]
    # Рассчитаем значения прибыли и добавим в список:
    values.append(profits(target_subsample, preds_subsample, BUDGET, UNIT_PROFIT))

# Преобразуем их в Series:
values = pd.Series(values)

# Рассчитаем верхний и нижний квантили:
lower = values.quantile(0.025)
upper = values.quantile(0.975)

# Найдем среднюю прибыль:
mean = values.mean()

# Определим риск убытков:
risk = (values<0).mean()

# Выведем полученные значения на экран:
print("Средняя прибыль:", mean)
print("2,5%-квантиль:", upper)
print("2,5%-квантиль:", lower)
print("Риск убытков:", risk)

Средняя прибыль: 425938526.9103001
2,5%-квантиль: 947976353.3924999
2,5%-квантиль: -102090094.8225
Риск убытков: 0.06


In [44]:
values = []

for i in range(1000):
    target_subsample = target_1_valid.sample(n=500, replace=True, random_state=state)
    preds_subsample = predictions_1_valid[target_subsample.index]
    values.append(profits(target_subsample, preds_subsample, BUDGET, UNIT_PROFIT))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()
risk = (values<0).mean()

print("Средняя прибыль:", mean)
print("2,5%-квантиль:", upper)
print("2,5%-квантиль:", lower)
print("Риск убытков:", risk)

Средняя прибыль: 518259493.6952
2,5%-квантиль: 953612982.1025
2,5%-квантиль: 128123231.4
Риск убытков: 0.003


In [45]:
values = []

for i in range(1000):
    target_subsample = target_2_valid.sample(n=500, replace=True, random_state=state)
    preds_subsample = predictions_2_valid[target_subsample.index]
    values.append(profits(target_subsample, preds_subsample, BUDGET, UNIT_PROFIT))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()
risk = (values<0).mean()

print("Средняя прибыль:", mean)
print("2,5%-квантиль:", upper)
print("2,5%-квантиль:", lower)
print("Риск убытков:", risk)

Средняя прибыль: 420194005.3432
2,5%-квантиль: 989629939.8475
2,5%-квантиль: -115852609.11749999
Риск убытков: 0.062


## Общий вывод

В результате применения техники Bootstrap к обученной модели,
риск убытков менее 2,5% демонстрирует только второй регион.
Таким образом, несмотря на самый низкий расчетный показатель прибыли и предсказанный объем сырья, для последующей разработки скважин стоит рекомендовать именно его.
